<h1><span style="color:red">Generate Contingency Tables</span></h1>

### This notebook reads numeric and categorical variables from the survey dataset, lets users compute a contingency table from variables of interest and test for independence.

## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

In [ ]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display
from ipyfilechooser import FileChooser

import pandas as pd    
import numpy as np
import panel as pn
import statsmodels.api as sm

pn.extension()
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

## 2. Read the survey file

In [ ]:
# read the csv file
df = panellibs.extract_data(absolutePath + csv_file)

# create a list of variable names
variables_df = pd.DataFrame({'varname':df.columns})
printmd("<b><span style='color:red'>All variables in the survey file:</span></b>")
print(variables_df.varname.values)

## 3. Select variables from survey

In [ ]:
# input csv
# df = pd.read_csv('aim1_aim2_cleansed.csv')
df = sm.datasets.get_rdataset("Arthritis", "vcd").data
df.head()

In [ ]:
# select number of variables for contingency table
n_row = pn.widgets.IntSlider(name='Select Number of Row Variables',start=1, end=2, value=1)
n_col = pn.widgets.IntSlider(name='Select Number of Column Variables',start=1, end=2, value=1)

pn.Column(n_row, n_col)

In [ ]:
# select variables from survey
row_vars = pn.widgets.MultiChoice(name='Select Row Variables for Contingency Table',
                                  value=[], options=list(df.columns), max_items=n_row.value)
col_vars = pn.widgets.MultiChoice(name='Select Column Variables for Contingency Table',
                                  value=[], options=list(df.columns), max_items=n_col.value)
pn.Row(row_vars, col_vars, height=250)

In [ ]:
# handle NaN values in survey/dataframe

## 4. Generate contingency table

In [ ]:
def generate_table(df, row_variables, column_variables):
    """
    Helper function to generate a contingency table from the input dataframe
    with a given list of row_variables and column_variables.
    """
    tab = pd.crosstab([df[row_variables[i]] for i in range(len(row_variables))],
                      [df[column_variables[i]] for i in range(len(column_variables))])
    table = sm.stats.Table(tab)
    return table.table_orig, table

In [ ]:
# bin all numerical variables in original df
df_binned = df.apply(pd.to_numeric, errors='ignore')
numeric_cols = df_binned.select_dtypes(include=np.number).columns.tolist()
df_binned[numeric_cols] = df_binned[numeric_cols].apply(pd.cut, bins=5)

In [ ]:
# generate contingency table
selected_row_vars = row_vars.value
selected_col_vars = col_vars.value
tab = generate_table(df_binned, selected_row_vars, selected_col_vars)
tab[0]

In [ ]:
# denote whether contingency table is two-way or multi-way
two_way = False
total_vars = len(row_vars.value + col_vars.value)
if total_vars == 2:
    two_way = True

## 5. Test for independence

In [ ]:
def chi_square(table, alpha=.05):
    """
    Performs a chi-square test of independence in an
    (r x c) contingency table at the given significance level
    """
    chi2 = table.test_nominal_association()
    
    # display a warning if expected counts are < 5
    exp_freq = table.fittedvalues.to_numpy()
    if (exp_freq < 5).sum() != 0:
        print("Warning: table contains expected frequencies less than 5.")

    # display results of chi-square test
    p_val = chi2.pvalue
    if p_val < alpha:
        print("P-value: {}. Table variables are associated at significance level: {}".format(p_val.round(4), alpha))
    else:
        print("P-value: {}. Table variables are independent at significance level: {}".format(p_val.round(4), alpha))

In [ ]:
# perform chi-square test of independence for two-way tables
if two_way:
    result = chi_square(tab[1])
result

In [ ]:
# add case for ordinal data

In [ ]:
# generalize for only r x c not stratified tables
# add check to see if variables are ordinal
# add check for NaN values in df